# Container N

In [3]:
'''
! mkdir -p /root/.pip


%%writefile /root/.pip/pip.conf
[global]
index-url=http://ftp.daumkakao.com/pypi/simple
trusted-host=ftp.daumkakao.com
'''

'\n! mkdir -p /root/.pip\n\n\n%%writefile /root/.pip/pip.conf\n[global]\nindex-url=http://ftp.daumkakao.com/pypi/simple\ntrusted-host=ftp.daumkakao.com\n'

In [3]:
!pip list

Package                 Version              Location
----------------------- -------------------- ------------------
absl-py                 0.14.1
albumentations          1.1.0
alembic                 1.7.5
appdirs                 1.4.4
argon2-cffi             21.1.0
astunparse              1.6.3
attrs                   21.2.0
audioread               2.1.9
autopage                0.4.0
backcall                0.2.0
batchgenerators         0.23
bleach                  4.1.0
cachetools              4.2.4
certifi                 2019.11.28
cffi                    1.15.0
chardet                 3.0.4
clang                   5.0
cliff                   3.9.0
cmaes                   0.8.2
cmd2                    2.3.0
colorlog                6.6.0
cycler                  0.10.0
dbus-python             1.2.16
debugpy                 1.5.0
decorator               5.1.0
defusedxml              0.7.1
dicom2nifti             2.3.0
entrypoints             0.3
flatbuffers             1.12
future 

In [2]:
import os

def maybe_mkdir_p(directory: str) -> None:
    os.makedirs(directory, exist_ok=True)

maic_dir = '/tf/backup/'
base_dir = os.path.join(maic_dir, 'working')
input_dir = '/mnt/dataset/'
temp_dir = '/tf/temp/'

maybe_mkdir_p(base_dir)
maybe_mkdir_p(temp_dir)

# ! git clone https://github.com/keemsir/nnUNet.git

respository_dir = os.path.join(maic_dir, 'nnUNet1')
os.chdir(respository_dir)

! pip install -e .

os.chdir(base_dir)

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
Obtaining file:///tf/backup/nnUNet1
     |████████████████████████████████| 151 kB 19.5 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 73.6 MB/s eta 0:00:01
     |████████████████████████████████| 48.4 MB 36.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 119.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 113.3 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 127.1 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 107.4 MB/s eta 0:00:01
  Created wheel for dicom2nifti: filename=dicom2nifti-2.3.0-py3-none-any.whl size=42932 sha256=2306a6899bd254ef8441cb84689d840a169a6db94072da8a337ba02f4ca4b54c
  Stored in directory: /root/.cache/pip/wheels/6c/63/4b/bfdb925ebf1c6ee721258d7f8a69951cd76e6d178bb0d0395b
  Created wheel for medpy: filename=MedPy-0.4.0-py3-none-any.whl size=214963 sha256=3abb89fca6edd5c86df2ee7879af189697df86cabd4870546c2c2fcb9ecd574b
  Sto

In [4]:
# installed
import shutil
import json
import numpy as np

from collections import OrderedDict
from scipy import special
import copy
import cv2
from tqdm import tqdm
import h5py

# must install
import pydicom
import nibabel as nib
import SimpleITK as sitk

In [6]:
task_name = 'Task50_PETCT'

main_dir = os.path.join(base_dir, 'nnUNet/nnunet')
mainT_dir = os.path.join(temp_dir, 'nnUNet/nnunet')

rawbase_dir = os.path.join(mainT_dir, 'nnUNet_raw_data_base/')

pp_dir = os.path.join(mainT_dir, 'preprocessed')
tasks_dir = os.path.join(mainT_dir, 'Tasks')
task_dir = os.path.join(tasks_dir, task_name)

model_dir = os.path.join(main_dir, 'nnUNet_trained_models')


# 1. Data preprocessing
maybe_mkdir_p(tasks_dir)
maybe_mkdir_p(temp_dir)

# 2. Directory
maybe_mkdir_p(main_dir)
maybe_mkdir_p(model_dir)
maybe_mkdir_p(pp_dir)


#Environment Setting
os.environ['nnUNet_raw_data_base'] = rawbase_dir #os.path.join(mainT_dir, 'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed'] = pp_dir #os.path.join(mainT_dir, 'preprocessed')
os.environ['RESULTS_FOLDER'] = model_dir #os.path.join(main_dir, 'nnUNet_trained_models')


print('Setting Completed!')

Setting Completed!


# 1-1. Data preprocessing

In [7]:
size_dic = {}
def hdf2nifti(hdf_folder: str, save_folder: str):
    # hdf_folder : [train_dir, test_dir] hdf5 file path
    # save_folder : [imagesTr, imagesTs] Save Folder path
    maybe_mkdir_p(os.path.join(save_folder, 'imagesTr'))
    maybe_mkdir_p(os.path.join(save_folder, 'labelsTr'))
    print('Creating "{}" Image & Label ..'.format(os.path.basename(os.path.normpath(save_folder))))
    hdf5_files = os.listdir(hdf_folder)


    for hdf5_file in hdf5_files:


        hdf5_path = os.path.join(hdf_folder, hdf5_file)

        # image
        f_i = h5py.File(hdf5_path, 'r')
        ctarr = np.asarray(f_i['CT'])
        petarr = np.asarray(f_i['PET'])
        labels = np.asarray(f_i['Aorta'])
        sizearr = np.asarray(f_i['Size'])
        f_i.close()
        
        size_dic[hdf5_file[:17]] = sizearr # new

        SLICE_SIZE_X, SLICE_SIZE_Y, SLICE_COUNT = ctarr.shape
        images = np.empty([SLICE_SIZE_X, SLICE_SIZE_Y, SLICE_COUNT, 0], dtype=np.single)

        image_ct = np.expand_dims(ctarr, axis=3)
        images = np.append(images, image_ct, axis=3)
        image_pet = np.expand_dims(petarr, axis=3)
        images = np.append(images, image_pet, axis=3)

        hdf5_file_NAME = hdf5_file
        
        af_x, af_y, af_z = size_dic[hdf5_file[:17]]
        affine_size = np.eye(4) * [af_x, af_y, af_z, 1]
        
        
        niim = nib.Nifti1Image(images, affine=affine_size)
        nib.save(niim, os.path.join(save_folder, 'imagesTr/{}.nii.gz'.format(hdf5_file[:17])))

        nila = nib.Nifti1Image(labels, affine=affine_size)
        nib.save(nila, os.path.join(save_folder, 'labelsTr/{}.nii.gz'.format(hdf5_file[:17])))
        print(hdf5_file, ' : ', size_dic[hdf5_file[:17]])


    print('"{}" Image & Label Completed !!'.format(os.path.basename(os.path.normpath(save_folder))))
    print('Image Patient : {}'.format(len(os.listdir(input_dir))))

hdf2nifti(input_dir, task_dir)

Creating "Task50_PETCT" Image & Label ..
23090580_20131226.hdf5  :  [4.6875     4.6875     3.27001953]
23090618_20161212.hdf5  :  [4.6875     4.6875     3.27001953]
23090628_20150204.hdf5  :  [4.6875     4.6875     3.27001953]
23010011_20150408.hdf5  :  [4.07283 4.07283 3.     ]
23090643_20121227.hdf5  :  [4.6875     4.6875     3.27001953]
23090644_20131216.hdf5  :  [4.6875     4.6875     3.27001953]
23090636_20121018.hdf5  :  [4.6875     4.6875     3.27001953]
23090585_20130213.hdf5  :  [4.6875     4.6875     3.27001953]
23010010_20150805.hdf5  :  [4.07283 4.07283 3.     ]
23170007_20190507.hdf5  :  [3.64583325 3.64583325 3.27001953]
23090572_20130226.hdf5  :  [4.6875     4.6875     3.27001953]
23010050_20130704.hdf5  :  [4.07283 4.07283 3.     ]
23010020_20141219.hdf5  :  [4.07283 4.07283 3.     ]
23010021_20141128.hdf5  :  [4.07283 4.07283 3.     ]
23090603_20141212.hdf5  :  [4.6875     4.6875     3.27001953]
23010047_20130904.hdf5  :  [4.07283 4.07283 3.     ]
23090601_20130225.hdf

In [8]:
def json_mk(save_dir: str):
    # Path
    imagesTr = os.path.join(save_dir, 'imagesTr')
    imagesTs = os.path.join(save_dir, 'imagesTs')
    maybe_mkdir_p(imagesTr)
    maybe_mkdir_p(imagesTs)

    overwrite_json_file = True
    json_file_exist = False

    if os.path.exists(os.path.join(save_dir, 'dataset.json')):
        print('dataset.json already exist!')
        json_file_exist = True

    if json_file_exist == False or overwrite_json_file:

        json_dict = OrderedDict()
        json_dict['name'] = "PETCT"
        json_dict['description'] = "Medical Image AI Challenge 2021"
        json_dict['tensorImageSize'] = "4D"
        json_dict['reference'] = "https://maic.or.kr/competitions/"
        json_dict['licence'] = "SNUH"
        json_dict['release'] = "18/10/2021"

        json_dict['modality'] = {
            "0": "CT",
            "1": "PET"
        }
        json_dict['labels'] = {
            "0": "background",
            "1": "Aorta"
        }

        train_ids = sorted(os.listdir(imagesTr))
        test_ids = sorted(os.listdir(imagesTs))
        json_dict['numTraining'] = len(train_ids)
        json_dict['numTest'] = len(test_ids)

        json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

        json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids] #(i[:i.find("_0000")])

        with open(os.path.join(save_dir, "dataset.json"), 'w') as f:
            json.dump(json_dict, f, indent=4, sort_keys=False)

        if os.path.exists(os.path.join(save_dir, 'dataset.json')):
            if json_file_exist == False:
                print('dataset.json created!')
            else:
                print('dataset.json overwritten!')
json_mk(task_dir)

dataset.json created!


# 1-2. Data preprocessing

In [9]:
os.listdir('/tf/temp/nnUNet/nnunet/Tasks/Task50_PETCT/')

['imagesTr', 'labelsTr', 'imagesTs', 'dataset.json']

In [10]:
# os.chdir(mainT_dir)
!nnUNet_convert_decathlon_task -i /tf/temp/nnUNet/nnunet/Tasks/Task50_PETCT -output_task_id 550 # -i : task_dir
!nnUNet_plan_and_preprocess -t 550 # --verify_dataset_integrity
# os.chdir(base_dir)



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

23010001_20170421
23010006_20151221
23010011_20150408
23010016_20141229
23010021_20141128
23010026_20140611
23010041_20131031
23010046_20130905
before crop: (2, 284, 200, 200) after crop: (2, 284, 173, 174) spacing: [3.        4.0728302 4.0728302] 

before crop: (2, 284, 200,

# 2. Training

In [11]:
# Model 1 (epoch 100, apply affine)

!nnUNet_train 3d_fullres nnUNetTrainerV2_Loss_CEGDL 550 0
!nnUNet_train 3d_fullres nnUNetTrainerV2_Loss_CEGDL 550 1
!nnUNet_train 3d_fullres nnUNetTrainerV2_Loss_CEGDL 550 2
!nnUNet_train 3d_fullres nnUNetTrainerV2_Loss_CEGDL 550 3
!nnUNet_train 3d_fullres nnUNetTrainerV2_Loss_CEGDL 550 4



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNet_variants.loss_function.nnUNetTrainerV2_Loss_CEGDL.nnUNetTrainerV2_Loss_CEGDL'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'CT', 1: 'PET'}
use_mask_for_norm OrderedDict([(0, False), (1, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'CT'), (1, 'nonCT')])
stages...

stage:  0
{'batch_size': 2, 'num_pool_per_axis': [4, 6, 4], 'patch_size': array